In [186]:
#Necessary libraries
import pandas as pd
import pickle  # Serialización deserialización de tipos de datos
from scipy.stats import poisson

In [187]:
with open('dict_grupos', 'rb') as g:
    dict_groups = pd.read_pickle(g)

df_fixture = pd.read_csv('fifa_worldcup_fixture.csv')

### Cálculo de la forateleza de un equipo en relación con sus anotaciones

In [188]:
cups = pd.read_csv('clean_data.csv')
df_home = cups[['Home', 'Home score', 'Away score']]
df_away = cups[['Away', 'Home score', 'Away score']]

df_home = df_home.rename(columns={'Home':'Team', 'Home score':'GoalsScored','Away score':'GoalsConceded'})
df_away = df_away.rename(columns={'Away':'Team', 'Home score': 'GoalsConceded','Away score':'GoalsScored'})

df_strength = pd.concat([df_home, df_away], ignore_index=True).groupby('Team').mean()
df_strength

,GoalsScored,GoalsConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.752941,1.164706
Australia,0.850000,1.850000
Austria,1.576923,1.692308
...,...,...
Uruguay,1.553571,1.303571
Wales,0.625000,1.250000
West Germany,2.050847,1.254237


In [189]:
df_strength.index

Index(['Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Belgium',
       'Bolivia', 'Bosnia and Herzegovina', 'Brazil', 'Bulgaria', 'Cameroon',
       'Canada', 'Chile', 'China', 'Colombia', 'Costa Rica', 'Croatia', 'Cuba',
       'Czech Republic', 'Czechoslovakia', 'Denmark', 'Dutch East Indies',
       'East Germany', 'Ecuador', 'Egypt', 'El Salvador', 'England',
       'FR Yugoslavia', 'France', 'Germany', 'Ghana', 'Greece', 'Haiti',
       'Honduras', 'Hungary', 'Iceland', 'Iran', 'Iraq', 'Israel', 'Italy',
       'Ivory Coast', 'Jamaica', 'Japan', 'Kuwait', 'Mexico', 'Morocco',
       'Netherlands', 'New Zealand', 'Nigeria', 'North Korea',
       'Northern Ireland', 'Norway', 'Panama', 'Paraguay', 'Peru', 'Poland',
       'Portugal', 'Qatar', 'Republic of Ireland', 'Romania', 'Russia',
       'Saudi Arabia', 'Scotland', 'Senegal', 'Serbia',
       'Serbia and Montenegro', 'Slovakia', 'Slovenia', 'South Africa',
       'South Korea', 'Soviet Union', 'Spain', 'Sweden', 'Sw

### Predicción Ganador


In [190]:
def predict_winner(home, away):
    if home in df_strength.index and away in df_strength.index:
        l_home = df_strength.at[home,'GoalsScored']*df_strength.at[away,'GoalsConceded']
        l_away = df_strength.at[away,'GoalsScored']*df_strength.at[home,'GoalsConceded']
        p_home, p_away, p_draw = 0, 0, 0

        for x in range(0,11):
            for y in range(0,11):
                p = poisson.pmf(x, l_home)*poisson.pmf(y,l_away)
                if x == y:
                    p_draw +=p
                elif x > y:
                    p_home +=p
                else:
                    p_away +=p
        
        points_home = 3*p_home + p_draw
        points_away = 3*p_away + p_draw
        return (points_home, points_away)
    else:
        return (0,0)

### Testeo de función

In [191]:
predict_winner('Senegal', 'Netherlands')

(0.6975349047221162, 2.126680652359251)

### Predicción de la copa

#### Fase de grupos

In [192]:
df_matches_48 = df_fixture[:48].copy()
df_matches_knockout = df_fixture[48:56].copy()
df_matches_quarter = df_fixture[56:60].copy()
df_matches_semifinal = df_fixture[60:62].copy()
df_matches_final = df_fixture[62:].copy()

In [ ]:
df_matches_48['home'] = df_matches_48['home'].str.strip()
for group in dict_groups:
    teams_group = dict_groups[group]['Team'].values
    six_groups = df_matches_48[df_matches_48['home'].isin(teams_group)]
    for i, row in six_groups.iterrows():
        home, away = row['home'], row['away']
        home = home.strip()
        away = away.strip()
        points_home, points_away = predict_winner(home, away)
        dict_groups[group].loc[dict_groups[group]['Team'] == home , 'Pts'] += points_home
        dict_groups[group].loc[dict_groups[group]['Team'] == away , 'Pts'] += points_away
    
    dict_groups[group] = dict_groups[group].sort_values('Pts', ascending=False).reset_index()
    dict_groups[group] = dict_groups[group][['Team', 'Pts']]
    dict_groups[group] = dict_groups[group].round(0)

In [194]:
dict_groups['Grupo D']


,Team,Pts
0,France,7.0
1,Denmark,5.0
2,Tunisia,3.0
3,Australia,2.0


#### Octavos

In [195]:
df_matches_knockout

,home,score,away,year
48,Winners Group A,Match 49,Runners-up Group B,2022
49,Winners Group C,Match 50,Runners-up Group D,2022
50,Winners Group D,Match 52,Runners-up Group C,2022
51,Winners Group B,Match 51,Runners-up Group A,2022
52,Winners Group E,Match 53,Runners-up Group F,2022
53,Winners Group G,Match 54,Runners-up Group H,2022
54,Winners Group F,Match 55,Runners-up Group E,2022
55,Winners Group H,Match 56,Runners-up Group G,2022


In [196]:
#Actualizar calendario de octavos
group_winner = dict_groups[group].loc[0, 'Team']
runners_up  = dict_groups[group].loc[0, 'Team']
df_matches_knockout.replace({'Winners Group A': dict_groups['Grupo A'].loc[0, 'Team'],'Runners-up Group B': dict_groups['Grupo B'].loc[1, 'Team']}, inplace=True)
df_matches_knockout.replace({'Winners Group C': dict_groups['Grupo C'].loc[0, 'Team'],'Runners-up Group D': dict_groups['Grupo D'].loc[1, 'Team']}, inplace=True)
df_matches_knockout.replace({'Winners Group D': dict_groups['Grupo D'].loc[0, 'Team'],'Runners-up Group C': dict_groups['Grupo C'].loc[1, 'Team']}, inplace=True)
df_matches_knockout.replace({'Winners Group B': dict_groups['Grupo B'].loc[0, 'Team'],'Runners-up Group A': dict_groups['Grupo A'].loc[1, 'Team']}, inplace=True)
df_matches_knockout.replace({'Winners Group E': dict_groups['Grupo E'].loc[0, 'Team'],'Runners-up Group F': dict_groups['Grupo F'].loc[1, 'Team']}, inplace=True)
df_matches_knockout.replace({'Winners Group G': dict_groups['Grupo G'].loc[0, 'Team'],'Runners-up Group H': dict_groups['Grupo H'].loc[1, 'Team']}, inplace=True)
df_matches_knockout.replace({'Winners Group F': dict_groups['Grupo F'].loc[0, 'Team'],'Runners-up Group E': dict_groups['Grupo E'].loc[1, 'Team']}, inplace=True)
df_matches_knockout.replace({'Winners Group H': dict_groups['Grupo H'].loc[0, 'Team'],'Runners-up Group G': dict_groups['Grupo G'].loc[1, 'Team']}, inplace=True)

df_matches_knockout['winner'] = '*'
df_matches_knockout

,home,score,away,year,winner
48,Netherlands,Match 49,United States,2022,*
49,Argentina,Match 50,Denmark,2022,*
50,France,Match 52,Poland,2022,*
51,England,Match 51,Ecuador,2022,*
52,Germany,Match 53,Belgium,2022,*
53,Brazil,Match 54,Uruguay,2022,*
54,Croatia,Match 55,Spain,2022,*
55,Portugal,Match 56,Switzerland,2022,*


In [197]:
def winner(df_knockout_upd):
    for index, row in df_knockout_upd.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_winner(home, away)
        if points_home > points_away:
            winr = home
        else:
            winr = away

        df_knockout_upd.loc[index, 'winner'] = winr

    return df_knockout_upd

In [198]:
winner(df_matches_knockout)

,home,score,away,year,winner
48,Netherlands,Match 49,United States,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Poland,2022,France
51,England,Match 51,Ecuador,2022,England
52,Germany,Match 53,Belgium,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Croatia,Match 55,Spain,2022,Spain
55,Portugal,Match 56,Switzerland,2022,Portugal


#### Cuartos de Final

In [199]:
df_matches_quarter

,home,score,away,year
56,Winners Match 53,Match 58,Winners Match 54,2022
57,Winners Match 49,Match 57,Winners Match 50,2022
58,Winners Match 55,Match 60,Winners Match 56,2022
59,Winners Match 51,Match 59,Winners Match 52,2022


In [200]:
def update_fixture(df_fixture_past, df_fixture_present):
    for index, row in df_fixture_past.iterrows():
        winr = df_fixture_past.loc[index,'winner']
        match = df_fixture_past.loc[index, 'score']
        df_fixture_present.replace({f'Winners {match}': winr}, inplace=True)
    df_fixture_present['winner'] = '*'
    return df_fixture_present

In [201]:
update_fixture(df_matches_knockout, df_matches_quarter)


,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,*
57,Netherlands,Match 57,Argentina,2022,*
58,Spain,Match 60,Portugal,2022,*
59,England,Match 59,France,2022,*


In [202]:
winner(df_matches_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,Argentina,2022,Netherlands
58,Spain,Match 60,Portugal,2022,Portugal
59,England,Match 59,France,2022,France


#### Semifinal

In [203]:
update_fixture(df_matches_quarter, df_matches_semifinal)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,*
61,France,Match 62,Portugal,2022,*


In [205]:
winner(df_matches_semifinal)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,Brazil
61,France,Match 62,Portugal,2022,France


### Final

In [206]:
update_fixture(df_matches_semifinal, df_matches_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,*
63,Brazil,Match 64,France,2022,*


In [210]:
#GANADOR
winner(df_matches_final)

print(f"El equipo ganador de la Copa del Mundo 2022 en Qatar es {df_matches_final.loc[63,'winner']} y el Subcampeón es {df_matches_final.loc[63,'away']}")

El equipo ganador de la Copa del Mundo 2022 en Qatar es Brazil y el Subcampeón es France
